In [34]:
import pandas as pd 
import numpy as np
import datetime
import time

In [35]:
data = pd.read_csv('kodigo_points.csv')

In [36]:
data['datestamp'] = pd.to_datetime(data['datestamp'])
data.dtypes

datestamp        datetime64[ns]
plateno                  object
geofence_name            object
dtype: object

In [37]:
data['track_period'] = data['datestamp'].dt.floor('30T')

In [38]:
data['time_start'] = data.groupby(['plateno','geofence_name','track_period']).transform('min')
data['time_end'] = data.groupby(['plateno','geofence_name','track_period']).transform('max')['datestamp']
data['count'] = data.groupby(['plateno','geofence_name','track_period']).transform('count')['datestamp']
data['duration'] = (data['time_end'] - data['time_start']).dt.total_seconds()


In [39]:
data = data.loc[(data['count'] > 5) & (data['duration'] > 300), :]
data = data.sort_values(by = ['plateno','geofence_name','track_period'])
data = data.loc[:, ['plateno','geofence_name','track_period','time_start','time_end','duration','count']]

In [40]:
data.drop_duplicates(inplace = True)
data = data.sort_values(by = ['plateno','track_period'])
data = data.sort_values(by = ['plateno','geofence_name'])

In [41]:
data['time_end_prev_loc'] = data.groupby(['plateno','geofence_name'])['time_end'].shift(1)
data['time_start_next_loc'] = data.groupby(['plateno','geofence_name'])['time_start'].shift(-1)


In [42]:
data['time_diff_next_loc'] = (data['time_start_next_loc'] - data['time_end']).dt.total_seconds()
data['time_diff_prev_loc'] = (data['time_start'] - data['time_end_prev_loc']).dt.total_seconds()

In [43]:
data['continue_next_loc']= data['time_diff_next_loc'] < 300
data['continue_prev_loc'] = data['time_diff_prev_loc'] < 300
data['continuing'] = data['continue_prev_loc'] & data['continue_next_loc']

In [44]:
data = data.loc[(data['continuing'] == False), :]

In [45]:
data['lead_time_diff_prev_loc'] = data.groupby(['plateno','geofence_name'])['time_diff_prev_loc'].shift(-1)
data['lead_time_end'] = data.groupby(['plateno','geofence_name'])['time_end'].shift(-1)

In [46]:
c1 = data['time_end_prev_loc'].isnull()
c2 = data['lead_time_diff_prev_loc'] < 300
c3 = data['time_diff_next_loc'] <= 300

condition = ((c1 | c2) & c3)

data['actual_time_end'] = np.where(condition, data['lead_time_end'], data['time_end'])

In [47]:
data['actual_time_end'] = np.where(data['time_diff_next_loc'].isnull(), data['time_end'], data['actual_time_end'])

In [48]:
data = data.loc[data['actual_time_end'].notnull(),:]
data = data.loc[:, ['plateno','geofence_name','time_start','actual_time_end']]

In [49]:
data['duration'] = (data['actual_time_end'] - data['time_start']).dt.total_seconds()

In [50]:
data['ranking'] = data.groupby(['plateno','geofence_name','actual_time_end'])['duration'].rank(ascending = False)

In [51]:
data = data.loc[data['ranking'] != 2, :]

In [52]:
data = data.loc[:, ['plateno','geofence_name','time_start','actual_time_end','duration']]

In [53]:
data = data.rename(columns= {'time_start':'datestamp_entry', 'actual_time_end': 'datestamp_left','duration' : 'dwell_time'}).reset_index(drop=True)

In [54]:
data = data.sort_values(by = 'datestamp_entry').reset_index(drop=True)

In [55]:
data['dwell_time_hms'] = pd.to_datetime(data['dwell_time'], unit='s').dt.strftime("%H:%M:%S")

### Normalizing CICO

In [56]:
data

,plateno,geofence_name,datestamp_entry,datestamp_left,dwell_time,dwell_time_hms
0,ABK1494,BICOL EXCL'T SLS & TRADING-CALAMBA,2021-03-01 00:00:58,2021-03-01 07:27:49,26811.0,07:26:51
1,CXK260,PH Transshipment Hub Nueva Ecija,2021-03-01 00:02:06,2021-03-01 05:56:10,21244.0,05:54:04
2,WQC386,SM Hypermarket - Ormoc,2021-03-01 06:14:57,2021-03-01 13:47:29,27152.0,07:32:32
3,CXK260,PH Transshipment Hub Nueva Ecija,2021-03-01 07:00:11,2021-03-01 08:22:11,4920.0,01:22:00
4,ABK1494,PH Batino Grocery - Source,2021-03-01 07:34:01,2021-03-01 16:16:45,31364.0,08:42:44
...,...,...,...,...,...,...
813,KAD4602,PH Cagayan Grocery - Destination,2021-03-31 18:05:10,2021-03-31 23:29:32,19462.0,05:24:22
814,CXK260,PH Transshipment Hub Nueva Ecija,2021-03-31 18:34:24,2021-03-31 18:58:25,1441.0,00:24:01
815,CXK260,PH Transshipment Hub Nueva Ecija,2021-03-31 19:04:25,2021-03-31 19:28:25,1440.0,00:24:00
816,CXK260,PH Transshipment Hub Nueva Ecija,2021-03-31 21:06:26,2021-03-31 22:28:27,4921.0,01:22:01


In [57]:
data = data.sort_values(by = ['plateno', 'datestamp_entry', 'geofence_name'])

In [58]:
data['prev_match'] = data['geofence_name'].eq(data['geofence_name'].shift(1))
data['next_match'] = data['geofence_name'].eq(data['geofence_name'].shift(-1))
data['continuing'] = np.where((data['prev_match']) & (data['next_match']), 1, 0)

data

,plateno,geofence_name,datestamp_entry,datestamp_left,dwell_time,dwell_time_hms,prev_match,next_match,continuing
0,ABK1494,BICOL EXCL'T SLS & TRADING-CALAMBA,2021-03-01 00:00:58,2021-03-01 07:27:49,26811.0,07:26:51,False,False,0
5,ABK1494,PH Batino Grocery - Destination,2021-03-01 07:34:01,2021-03-01 16:16:45,31364.0,08:42:44,False,False,0
4,ABK1494,PH Batino Grocery - Source,2021-03-01 07:34:01,2021-03-01 16:16:45,31364.0,08:42:44,False,False,0
16,ABK1494,BICOL EXCL'T SLS & TRADING-CALAMBA,2021-03-01 16:32:11,2021-03-02 03:47:18,40507.0,11:15:07,False,False,0
21,ABK1494,PUREGOLD PRICE CLUB - PARIAN JR,2021-03-02 03:32:06,2021-03-02 03:43:09,663.0,00:11:03,False,False,0
...,...,...,...,...,...,...,...,...,...
471,WQC386,PH Mandaue Grocery - Destination,2021-03-18 21:51:05,2021-03-19 03:06:32,18927.0,05:15:27,False,False,0
472,WQC386,PH Mandaue Grocery - Source,2021-03-18 21:52:06,2021-03-19 02:59:58,18472.0,05:07:52,False,False,0
507,WQC386,Puregold Price Club - Kananga,2021-03-20 01:43:11,2021-03-20 12:24:49,38498.0,10:41:38,False,False,0
545,WQC386,PH Mandaue Grocery - Destination,2021-03-21 08:38:04,2021-03-21 10:59:59,8515.0,02:21:55,False,False,0


In [59]:
data = data.loc[:, ['plateno', 'geofence_name','datestamp_entry', 'datestamp_left', 'dwell_time','dwell_time_hms', 'continuing']]
data = data.loc[data['continuing'] == 0]

In [60]:
data['has_same_next'] =  np.where(data['geofence_name'].eq(data['geofence_name'].shift(-1)), 1, 0)

In [61]:
data

,plateno,geofence_name,datestamp_entry,datestamp_left,dwell_time,dwell_time_hms,continuing,has_same_next
0,ABK1494,BICOL EXCL'T SLS & TRADING-CALAMBA,2021-03-01 00:00:58,2021-03-01 07:27:49,26811.0,07:26:51,0,0
5,ABK1494,PH Batino Grocery - Destination,2021-03-01 07:34:01,2021-03-01 16:16:45,31364.0,08:42:44,0,0
4,ABK1494,PH Batino Grocery - Source,2021-03-01 07:34:01,2021-03-01 16:16:45,31364.0,08:42:44,0,0
16,ABK1494,BICOL EXCL'T SLS & TRADING-CALAMBA,2021-03-01 16:32:11,2021-03-02 03:47:18,40507.0,11:15:07,0,0
21,ABK1494,PUREGOLD PRICE CLUB - PARIAN JR,2021-03-02 03:32:06,2021-03-02 03:43:09,663.0,00:11:03,0,0
...,...,...,...,...,...,...,...,...
471,WQC386,PH Mandaue Grocery - Destination,2021-03-18 21:51:05,2021-03-19 03:06:32,18927.0,05:15:27,0,0
472,WQC386,PH Mandaue Grocery - Source,2021-03-18 21:52:06,2021-03-19 02:59:58,18472.0,05:07:52,0,0
507,WQC386,Puregold Price Club - Kananga,2021-03-20 01:43:11,2021-03-20 12:24:49,38498.0,10:41:38,0,0
545,WQC386,PH Mandaue Grocery - Destination,2021-03-21 08:38:04,2021-03-21 10:59:59,8515.0,02:21:55,0,0


In [62]:
data['new_datestamp_left'] = np.where(data['has_same_next'] == 1, data['datestamp_left'].shift(-1), data['datestamp_left'])

In [63]:
data['to_remove'] = data['has_same_next'].shift(1) == 1
data = data.loc[data['to_remove'] == False, ['plateno', 'geofence_name', 'datestamp_entry','new_datestamp_left' ,'dwell_time', 'dwell_time_hms']]

In [64]:
data.rename(columns= {'new_datestamp_left' :'datestamp_left'}, inplace=True)

In [65]:
data['dwell_time'] = (data['datestamp_left'] - data['datestamp_entry']).dt.total_seconds()
data['dwell_time_hms'] = pd.to_datetime(data['dwell_time'], unit='s').dt.strftime("%H:%M:%S")
data.reset_index(drop = True, inplace = True)